## Load documents from GIT Repository
Langchain `DocumentLoaders` load data into the standard LangChain `Document` format. We can use components provided by langchain or any custom loader implementing `DocumentLoader` interface.

Some of available loaders in [langchain_community.document_loaders](https://python.langchain.com/docs/integrations/document_loaders/):
- **Webpages**: allow you to load webpages.
- **PDFs**: allow you to load PDF documents.
- **Cloud Providers**: allow you to load documents from your favorite cloud providers.
- **Tools**: allow you to load data from commonly used tools like Slack, Quip, Github and more..
- **Local**: allow you to load data from your local file system.

In order to load files from GIT repository we can use `GitLoader` community package that will allow to load reposiitory data, where each document would represent one file in the repository.

We can also utilize `GitPython` package to clone repository and retreive another useful information.

```
> pip install --upgrade --quiet  GitPython
```


In [ ]:
from typing import Iterator
from git import Repo
from langchain_core.documents import Document
from langchain_community.document_loaders import GitLoader

jsLoader = GitLoader(repo_path="./client-server-example/", branch='master', file_filter=lambda file_path: file_path.endswith(".js"))
mdLoader = GitLoader(repo_path="./client-server-example/", branch='master', file_filter=lambda file_path: file_path.endswith(".md"))
defaultLoader = GitLoader(repo_path="./client-server-example/", branch='master', file_filter=lambda file_path: not file_path.endswith(".md") and not file_path.endswith(".js"))

jsData = jsLoader.load()
mdData = mdLoader.load()
otherData = defaultLoader.load()

print(len(jsData))
print(len(mdData))
print(len(otherData))

#print(mdData[0])
#print(jsData[1])
#print(otherData[3])

repo = Repo("./client-server-example/")
def get_changed_files():
    changed_files = []
    diff_index = repo.index.diff(None)
    for diff_item in diff_index:
        changed_files.append(diff_item.a_path)

    return changed_files

def get_commits_from_branch(branch_name):
    branch = next(filter(lambda b: b.name == branch_name, repo.branches), None)
    if (branch is not None):        
        commits = list(branch.commit.iter_items(repo, branch.commit))    
        return [commit.message for commit in commits]
    ref = next(filter(lambda b: b.name == f"origin/{branch_name}", repo.remote().refs), None)
    if ref is not None:        
        commits = list(repo.iter_commits(ref))    
        return [commit.message for commit in commits]    
    return []

def load_commits() -> Iterator[Document]:
    for commit in repo.iter_commits():
        metadata = {
            "commit_author_name": commit.author.name,
            "commit_author_email": commit.author.email,
            "commit_authored_datetime": commit.authored_datetime,
            "commit_committed_datetime": commit.committed_datetime,
        }
        yield Document(page_content=commit.message, metadata=metadata)

commitsData = list(load_commits())

print(len(commitsData))
#print(commitsData[0])

## Tokenize and Split documents into chunks

There are different ways to split a document into chunks, the most challenging part that we need to consider is to split the document into meaningful chunks, and it can strongly depend on the document type and the use case.

In our case there are 3 types of documents we can consider:
  - **Text documents**: We can split the text into paragraphs, sentences, or words.
  - **Code documents**: We can split the code into functions, classes, or lines.
  - **Git data**: We can split the git data into commits, files, or lines.

Splitters supported by [langchain.text_splitter](https://python.langchain.com/v0.2/api_reference/text_splitters/index.html) that we can use:
  - **RecursiveCharacterTextSplitter**: Implementation of splitting text that recursively looks at characters.
  - **MarkdownHeaderTextSplitter**: Implementation of splitting markdown files based on specific headers.
  - **TokenTextSplitter**: Implementation of splitting text that looks at tokens.
  - **SentenceTransformersTokenTextSplitter**: It is a specialized text splitter for use with the sentence-transformer models. The default behaviour is to split the text into chunks that fit the token window of the sentence transformer model that you would like to use.
  - **RecursiveJsonSplitter**: Implementation of splitting text that looks at characters. Recursively tries to split by different characters to find the one that works.
  - **Language**: for CPP, Python, Ruby, etc...


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, Language, TokenTextSplitter, SentenceTransformersTokenTextSplitter

js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=100, chunk_overlap=0
)
md_splitter = MarkdownTextSplitter(chunk_size=100, chunk_overlap=10)
token_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)
sentence_splitter = SentenceTransformersTokenTextSplitter()

js_docs = js_splitter.split_documents(jsData)
md_docs = md_splitter.split_documents(mdData)
other_docs = token_splitter.split_documents(otherData)
commit_docs = sentence_splitter.split_documents(commitsData)


print(len(commit_docs))
print(len(js_docs))
print(len(md_docs))
print(len(other_docs))

#print(js_docs[0])
#print(md_docs[0])
#print(other_docs[0])
#print(commit_docs[0])


## Create embedding vectors

Embeddings are essential for LLM tasks, they are high-dimensional vectors that capture the semantic meaning of tokens in chunks. We will use them for document corpus and for the query to search for relevant chunks that will be included into the context to generate completions.

From this point we need to find a structure of contextual query to find proper documents for suggestion. Lets assume it will be changes in the code of current document. We can use `GitPython` to get the changes in the repository and use them as a query.

To understand embeddings and how they aligned you can play with text embeddings in Cohere Playground (https://dashboard.cohere.com/playground/embed)


In [ ]:
import asyncio
import numpy
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from sentence_transformers import SentenceTransformer

openAIEmb = OpenAIEmbeddings(api_key="KEY")
ollamaEmb = OllamaEmbeddings(model="starcoder2:3b")
hfEmb = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') #sentence-transformers/all-mpnet-base-v2
sbertEmb = SentenceTransformer('all-MiniLM-L6-v2')

(js_embeddings, md_embeddings) = await asyncio.gather(
    openAIEmb.aembed_documents([doc.page_content for doc in js_docs]),
    ollamaEmb.aembed_documents([doc.page_content for doc in md_docs])
    )

commit_embeddings = sbertEmb.encode([doc.page_content for doc in commit_docs], convert_to_tensor=True) # hfEmb.embed_documents(md_docs)

print(len(js_embeddings))
print(len(md_embeddings))
print(len(commit_embeddings))

# print vector dimensions
print(len(js_embeddings[0]))
print(len(md_embeddings[0]))
print(commit_embeddings.shape)

# understanding embeddings
# https://dashboard.cohere.com/playground/embed
# mother, father, aunt, uncle
plur = numpy.subtract(openAIEmb.embed_query("students"), openAIEmb.embed_query("student"))

print(numpy.dot(plur, openAIEmb.embed_query("cat")))
print(numpy.dot(plur, openAIEmb.embed_query("cats")))

## Load data to Vector DB
Vector DBs are very diverse, they support different types of embedding models and different types of search and API capabilities. Most of them support `langchain` models.



In [ ]:
# Simple In-Memory
# Chroma
# Faiss
# Qdrant

import faiss
import uuid
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

in_memory_vector_store = InMemoryVectorStore(openAIEmb)

faiss_vec_dim = commit_embeddings.shape[1] # vector size of size of sentence-BERT embeddings
faiss_index = faiss.IndexFlatL2(faiss_vec_dim)
faiss_index.train(commit_embeddings)
faiss_vector_store = FAISS(
    embedding_function=hfEmb,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

client = QdrantClient(":memory:")
client.create_collection(
    collection_name="text_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE), # 1536 - vector size of 'text-embedding-ada-002' OpenAI embeddings
)
client.create_collection(
    collection_name="text_collection_hf",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE), # 768 - vector size of SBERT embeddings
)
client.create_collection(
    collection_name="code_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE), # 1536 - vector size of 'text-embedding-ada-002' OpenAI embeddings
)
client.create_collection(
    collection_name="code_collection_sc",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE), # 3072 - default vector size of starcoder2 embeddings
)
client.create_collection(
    collection_name="docs_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE), # 1536 - vector size of 'text-embedding-ada-002' OpenAI embeddings
)
qdrant_vector_store = QdrantVectorStore(
    client=client,
    collection_name="docs_collection",
    embedding=openAIEmb, #ollamaEmb
)

# Add documents to the vector stores
in_memory_vector_store.add_documents(documents=js_docs)

qdrant_vector_store.add_documents(documents=md_docs)

uuids = [str(uuid.uuid4()) for _ in range(len(commit_docs))]
faiss_vector_store.add_documents(documents=commit_docs, ids=uuids)
# FAISS.from_documents()

## Search vectorized data
In `Langchain`, components that are responsible to returns documents given an unstructured query ara called `Retreivers`, responsible for finding the most relevant documents along with its "relativity" to the query if possible.

Retreivers, availabl for `langchain` integration can be found here https://python.langchain.com/docs/integrations/retrievers/ along with the usage guide https://python.langchain.com/docs/how_to/#retrievers

We will start from vectorized data search capabilities based on vector store-backed retriever and search specific to certain types of stores.

In [ ]:
# semantic similarity SBERT (sbert api)
# similarity search by query and by vector
# use retreivers without Vector DB (KNN, etc.)
# FAISS similarity, index search (faiss api)
# Qdrant MMR
# Chroma

import torch
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import BM25Retriever
from sentence_transformers import SimilarityFunction
from sentence_transformers import util
from langchain_qdrant import QdrantVectorStore
from tree_sitter_languages import get_parser

current_diff = repo.git.diff()
#print(current_diff)

js_retriever = in_memory_vector_store.as_retriever(k=3) # similarity, mmf, scored similarity
# js_retriever = in_memory_vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}, k=1)
similar_js_docs = js_retriever.invoke(current_diff)
print("similar js docs")
print(similar_js_docs)

# summarize current diff to search info in commits
chat = ChatOpenAI(model="gpt-4o-mini", api_key="KEY")
summary_template = ChatPromptTemplate.from_template("Create short meaningful commit message of git diff.\n{git_diff}")
# Try FewShotChatMessagePromptTemplate with example prompts
summary_chain = summary_template | chat | StrOutputParser()
summary = summary_chain.invoke({"git_diff":current_diff})
print(f"summary: {summary}")

# similarity search by query, vector and score
summary_embeddings = hfEmb.embed_query(summary)
similar_commits = faiss_vector_store.similarity_search_by_vector(summary_embeddings) # ANN search
print("FAISS similar commits")
print(similar_commits)

similar_commits_score = faiss_vector_store.similarity_search_with_score(summary, k=2)
print("FAISS similar commits with score")
for (doc, score) in similar_commits_score:
    print(f"[{score}] {doc.page_content}")

# specialized search with retreiver
# BM25, TF-IDF, KNN, SVM, etc.
remote_commits = get_commits_from_branch("with_router")
bm25_results = BM25Retriever.from_texts(remote_commits).invoke(summary)
print("BM25 similar commits")
print(bm25_results)

# SBERT smantic search (ENN search)
# Select model depending on Symmetric vs Asymmetric Semantic Search
sbertEmb.similarity_fn_name = SimilarityFunction.DOT_PRODUCT # SimilarityFunction.COSINE, etc.
# model processing options can be cpu or gpu
commits_emb = sbertEmb.encode(remote_commits, convert_to_tensor=True)
summary_emb = sbertEmb.encode([summary], convert_to_tensor=True)
commit_similarities = sbertEmb.similarity(summary_emb, commits_emb)[0] # pairwise_similarity
scores, indices = torch.topk(commit_similarities, k=2)
print("SBERT similar commits")
for score, idx in zip(scores, indices):
    print(f"[{score:.4f}] {remote_commits[idx]}".rstrip())

# speed optimization
commits_emb_gpu = commits_emb.to("cuda")
commits_emb_gpu = util.normalize_embeddings(commits_emb_gpu) # ENN
summary_emb_gpu = summary_emb.to("cuda")
summary_emb_gpu = util.normalize_embeddings(summary_emb_gpu)
hits = util.semantic_search(summary_emb_gpu, commits_emb_gpu, score_function=util.dot_score)
print("GPU similar commits")
print(hits)

# Qdrant MMR
md_results = qdrant_vector_store.max_marginal_relevance_search(current_diff, k=3)
print("Qdrant MMR similar md docs")
print(md_results)

# FAISS specific search with performance improvements
new_index = faiss.IndexIVFFlat(faiss_index, commit_embeddings.shape[1], 2)
new_index.train(commit_embeddings)
new_index.add(commit_embeddings)
#d,i = new_index.search(summary_emb, k=1)
print("FAISS specific search")
#print(i)

# understanding relevance, score
qdrant_test_store = QdrantVectorStore(
    client=client,
    collection_name="text_collection",
    embedding=openAIEmb
)

qdrant_test_store.add_texts(["She deposited money at the bank.","The boat was tied to the river bank."])

bank_relevance = qdrant_test_store.similarity_search_with_relevance_scores("steep bank")
bank_score = qdrant_test_store.similarity_search_with_score("steep bank")
bank_mmr = qdrant_test_store.max_marginal_relevance_search("steep bank")

print(bank_relevance)
print(bank_score)
print(bank_mmr)

bank_relevance = qdrant_test_store.similarity_search_with_relevance_scores("reliable bank")
bank_score = qdrant_test_store.similarity_search_with_score("reliable bank")
bank_mmr = qdrant_test_store.max_marginal_relevance_search("reliable bank")

print(bank_relevance)
print(bank_score)
print(bank_mmr)

# code similarity
sort = """
void f(int[] array) {
    boolean swapped = true;
    for (int i = 0; i < array.length && swapped; i++) {
        swapped = false;
        for (int j = 0; j < array.length - 1 - i; j++) {
           if (array[j] > array[j+1]) {
               int temp = array[j];
               array[j] = array[j+1];
               array[j+1]= temp;
               swapped = true;
           }
        }
    }
}
"""

stddev = """
const f = (trials, len) => {
    let sum = 0;
    let squ = 0.0;

    for (let i = 0; i < len; i++) {
        let d = trials[i];
        sum += d;
        squ += d * d;
    }

    let x = squ - (sum * sum) / len;
    let res = Math.sqrt(x / (len - 1));

    return Math.floor(res);
};
"""

indexOf = """
function f(array, value) {
    for (let i = 0; i < array.length; i++) {
        if (array[i] === value) {
            return i;
        }
    }
    return -1;
}
"""

sql_query = "SELECT * FROM customers WHERE city = 'Berlin'"

qdrant_code_store = QdrantVectorStore(
    client=client,
    collection_name="code_collection_sc",
    embedding=ollamaEmb
)

qdrant_code_store.add_texts([sort, stddev, indexOf])

qdrant_asm_store = QdrantVectorStore(
    client=client,
    collection_name="code_collection",
    embedding=openAIEmb
)

sort_relevance = qdrant_code_store.similarity_search_with_relevance_scores("sort")
sort_score = qdrant_code_store.similarity_search_with_score("sort")

print(sort_relevance)
print(sort_score)

sql_relevance = qdrant_code_store.similarity_search_with_relevance_scores(sql_query)
sql_score = qdrant_code_store.similarity_search_with_score(sql_query)

print(sort_relevance)
print(sort_score)

# code similarity with tree-sitter AST
js_parser = get_parser('javascript')
sql_parser = get_parser('sql')
java_parser = get_parser('java')

ts_query = sql_parser.parse(sql_query.encode()).root_node.sexp()

ts_indexOf = js_parser.parse(indexOf.encode()).root_node.sexp()
ts_stdev = js_parser.parse(stddev.encode()).root_node.sexp()
ts_sort = java_parser.parse(sort.encode()).root_node.sexp()

qdrant_code_store.add_documents([
    Document(page_content=ts_sort, metadata={'source':'ts_sort', 'source_code': sort, 'language':'java'}), 
    Document(page_content=ts_stdev, metadata={'source':'ts_stdev', 'source_code': stddev, 'language':'js'}), 
    Document(page_content=ts_indexOf, metadata={'source':'ts_indexOf', 'source_code': indexOf, 'language':'js'})])

ts_relevance = qdrant_code_store.similarity_search_with_relevance_scores(ts_query)
ts_score = qdrant_code_store.similarity_search_with_score(ts_query)
print(ts_relevance)
print(ts_score)



## Add filtering by metadata
Filtering by metadata is a common use case in search engines, we can utilize any specific data store or database api to filter the data, or we can use `langchain` retrievers capabilities to filter the data based on metadata.

In [ ]:
# filter with self query filter
# time-weighted retriever
# get doc from shunks

#from langchain_chroma import Chroma

file_list = get_changed_files()
print(file_list)
other_docs_store = InMemoryVectorStore.from_documents(other_docs, openAIEmb) # Chroma.from_documents(other_docs, openAIEmb)
other_results = other_docs_store.as_retriever(search_kwargs= {"filter": {"file_name": {"$in": file_list}}}).invoke(current_diff)

## Use text search approaches

In [ ]:
# Qdrant Hybrid
# Git search and embedding

## Use reranking for better results

In [ ]:
# ColBERT
# Cohere reranking

## Advanced search options
Logical and Semantic routing (different data source based on input) - https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb 
Query Analysis (to apply fiter based on input) - https://python.langchain.com/docs/tutorials/query_analysis/#query-analysis 
MultiQuery (LLM to analyze input and generate query) - https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb 

In [ ]:
# Logical and Semantic routing for different doc types
# Multi-Query query ewith retreiver
# Timestamp-weighted (based on commits)

## Prompt
The final prompt will be a combination of the context and the query. Its structure is strongly dependent on the use case, the data we have and LLM that is used. For example for completions we can use special purpose  **Stable Code** models that will require quite strict structure of the prompt. For general purpose models we can follow some common suggestions:
  - Be specific (dierectly specify the language, purpose, context, desired output)
  - Use examples (provide examples of the code suggestions, the data)
  - Use structured output if needed and possible
  - Provide system instructions if possible, use role-based prompts

### Defining Context
When defining context the main limitation is number of tokens we can provide to the model. 


## GraphRAG
https://microsoft.github.io/graphrag/ 